In [1]:
import sys
sys.path.append('/host/d/Github/')  # add the path to your own Example_UNet folder
import numpy as np
import nibabel as nb 
import pandas as pd
import os
import copy
import random
import Diffusion_denoising_thin_slice.simulation.ct_basic_PCD as ct
import Diffusion_denoising_thin_slice.functions_collection as ff
import ct_projector.projector.numpy.parallel as ct_para


main_path = '/host/d/Data/low_dose_CT/'  # change to your own data path

def split_angles_indexes(total_angles):
    arr = np.arange(total_angles)
   
    pairs = arr.reshape(-1, 2)  

    A = []
    B = []

    for pair in pairs:
        idx = random.randint(0, 1)  
        A.append(pair[idx])
        B.append(pair[1 - idx])

    A = np.array(A)
    B = np.array(B)
    return A,B

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# define the patient list
patient_sheet = ff.find_all_target_files(['nii_imgs/*'],main_path)
print('Found {} patients'.format(len(patient_sheet)))

Found 10 patients


In [ ]:
split_angle = True
for i in range(0, len(patient_sheet)):
    patient_id = os.path.basename(patient_sheet[i])

    print(patient_id)

    save_folder_case = os.path.join(main_path,'simulation_v1', patient_id)
    ff.make_folder([os.path.dirname(save_folder_case), save_folder_case])


    # img file
    img_file = os.path.join(patient_sheet[i],'img.nii.gz')
    print(img_file)
    # load img
    img_clean = nb.load(img_file).get_fdata().astype(np.float32)
    img_clean[img_clean < -1024] = -1024
    spacing = nb.load(img_file).header.get_zooms()[::-1]
    affine = nb.load(img_file).affine
    print('img shape, min, max: ', img_clean.shape, np.min(img_clean), np.max(img_clean))
    print('spacing: ', spacing)

    for noise_type in ['possion', 'gaussian']:
        # by vusialization we decided the following dose range
        # for MGH brain CT dataset
        # possion_hann_dose_range = [0.10,0.20]
        # gaussian_custom_dose_range = [0.15,0.25]

        # for mayo low does CT dataset
        dose_range = [0.15, 0.35]
        
        for k in range(0,2):
            save_folder_k = os.path.join(save_folder_case, noise_type+'_random_'+str(k));ff.make_folder([save_folder_k])

            if os.path.isfile(os.path.join(save_folder_k,'recon_odd.nii.gz')):
                print('already done, continue')
                continue

            # if noise_type == 'possion':
            #     dose_factor = np.random.uniform(possion_hann_dose_range[0],possion_hann_dose_range[1] + 1e-8)
            #     print('dose factor: ', dose_factor)
            # elif noise_type == 'gaussian':
            #     dose_factor = np.random.uniform(gaussian_custom_dose_range[0],gaussian_custom_dose_range[1] + 1e-8)

            if os.path.isfile(os.path.join(save_folder_k,'dose_factor.txt')):
                with open(os.path.join(save_folder_k,'dose_factor.txt'),'r') as f:
                    dose_factor = float(f.read())
                print('dose factor loaded: ', dose_factor)
            else:
                dose_factor = np.random.uniform(dose_range[0],dose_range[1] + 1e-8)
                # save into a txt file
                with open(os.path.join(save_folder_k,'dose_factor.txt'),'w') as f:
                    f.write('{:.4f}'.format(dose_factor))
            print('dose factor: ', dose_factor)


            # process img
            img0 = img_clean.copy()
            img0 = np.rollaxis(img0,-1,0)
      
            # define projectors
            geometry_file = '/host/d/Github/Diffusion_denoising_thin_slice/help_data/pcd_parallel_6x5_512.cfg'  # change to your own path
            projector = ct.define_forward_projector_pcd(img0,spacing, file_name = geometry_file)
            # FP
            # set angles
            angles = projector.get_angles()

            recon_noise = np.zeros((img0.shape[1], img0.shape[2], img0.shape[0]), np.float32)
            if split_angle:
                recon_noise_odd = np.zeros((img0.shape[1], img0.shape[2], img0.shape[0]), np.float32)
                recon_noise_even = np.zeros((img0.shape[1], img0.shape[2], img0.shape[0]), np.float32)

            for slice_n in range(0, img0.shape[0]):
                img_slice = img0[[slice_n],:,:].copy()
                img_slice = (img_slice[np.newaxis, ...] + 1000) / 1000 * 0.019 
                
                # forward proj
                prjs = ct_para.distance_driven_fp(projector, img_slice, angles)
                # ramp filter for image
                fprjs = ct_para.ramp_filter(projector, prjs, 'rl')

                # add noise
                if noise_type[0:2] == 'po':
                    # add poisson noise
                    noise_of_prjs = ct.add_poisson_noise(prjs, N0=1000000, dose_factor = dose_factor) - prjs
                elif noise_type[0:2] == 'ga':
                    # add gaussian noise
                    noise_of_prjs = ct.add_gaussian_noise(prjs, N0=1000000, dose_factor = dose_factor) - prjs

                # recon (not split)
                if split_angle == False:
                    if noise_type[0:2] == 'po':
                        # hann filter for noise projs
                        fnoise = ct_para.ramp_filter(projector, noise_of_prjs, 'hann')

                        # bp for noise
                        recon_hann_noise = ct_para.distance_driven_bp(projector, fnoise, angles, True) 
                    
                        # bp for image
                        recon_hann_image = ct_para.distance_driven_bp(projector, fprjs, angles, True)
                    
                        # final recon
                        recon_hann = (recon_hann_noise +recon_hann_image)[0,0] / 0.019 * 1000 - 1000
                    
                        
                        recon_noise[:,:,slice_n] = recon_hann
                        
                    elif noise_type[0:2] == 'ga':
                        # custom filter
                        custom_filter_file = '/host/d/Github/Diffusion_denoising_thin_slice/help_data/softTissueKernel_65'  # change to your own path
                        custom_additional_filter = ct.get_additional_filter_to_rl(custom_filter_file, projector.nu, projector.du, projector.nview)
                        # bp for noise 
                        recon_custom_noise = ct.interleave_filter_and_recon(projector, noise_of_prjs, custom_additional_filter,angles) 
                        # bp for image
                        recon_custom_image = ct.interleave_filter_and_recon(projector, fprjs, custom_additional_filter, angles)
                        # final recon
                        recon_custom = (recon_custom_noise + recon_custom_image)/ 0.019 * 1000 - 1000

                        recon_noise[:,:,slice_n] = recon_custom

                # recon (split)
                else:
                    if os.path.isfile(os.path.join(save_folder_k,'odd_even_indexes.npy')):
                        odd_indexes = np.load(os.path.join(save_folder_k,'odd_even_indexes.npy'), allow_pickle=True).item()['odd']
                        even_indexes = np.load(os.path.join(save_folder_k,'odd_even_indexes.npy'), allow_pickle=True).item()['even']
                    else:
                        odd_indexes, even_indexes = split_angles_indexes(len(angles))
                        # save angles indexes for reference
                        np.save(os.path.join(save_folder_k,'odd_even_indexes.npy'), {'odd':odd_indexes, 'even':even_indexes})
      

                    if noise_type[0:2] == 'po':
                        # hann filter for noise projs
                        fnoise = ct_para.ramp_filter(projector, noise_of_prjs, 'hann')

                        # split fnoise and fprjs to odd and even
                        fprjs_odd = fprjs[:, odd_indexes, ...]
                        fprjs_even = fprjs[:, even_indexes, ...]
                        fnoise_odd = fnoise[:, odd_indexes, ...]
                        fnoise_even = fnoise[:, even_indexes, ...]
                
                        # bp for image
                        recon_hann_image_odd = ct_para.distance_driven_bp(projector, fprjs_odd, angles[odd_indexes], True)
                        recon_hann_image_even = ct_para.distance_driven_bp(projector, fprjs_even, angles[even_indexes], True)

                        # bp for noise
                        recon_hann_noise_odd = ct_para.distance_driven_bp(projector, fnoise_odd, angles[odd_indexes], True)
                        recon_hann_noise_even = ct_para.distance_driven_bp(projector, fnoise_even, angles[even_indexes], True)

                        # final recon
                        recon_hann_odd = (recon_hann_image_odd + recon_hann_noise_odd)[0,0]  * 2 / 0.019 * 1000 - 1000
                        recon_hann_even = (recon_hann_image_even + recon_hann_noise_even)[0,0]  * 2 / 0.019 * 1000 - 1000
                        recon_noise_odd[:,:,slice_n] = recon_hann_odd
                        recon_noise_even[:,:,slice_n] = recon_hann_even

                    elif noise_type[0:2] == 'ga':
                        # custom filter
                        custom_filter_file = '/host/d/Github/Diffusion_denoising_thin_slice/help_data/softTissueKernel_65'  # change to your own path
                        custom_additional_filter = ct.get_additional_filter_to_rl(custom_filter_file, projector.nu, projector.du, projector.nview)

                        fprjs, projector_new = ct.interleave_filter_and_recon(projector, prjs, custom_additional_filter, angles, get_recon=False)
                        fnoise, _ = ct.interleave_filter_and_recon(projector, noise_of_prjs, custom_additional_filter, angles, get_recon=False)
                        
                        # split fprjs and fnoise to odd and even
                        fprjs_odd = fprjs[:, odd_indexes, ...]
                        fprjs_even = fprjs[:, even_indexes, ...] 
                        fnoise_odd = fnoise[:, odd_indexes, ...]
                        fnoise_even = fnoise[:, even_indexes, ...]
                        
                        # # bp for image
                        recon_custom_image_odd = ct_para.pixel_driven_bp(projector_new, fprjs_odd, angles[odd_indexes])
                        recon_custom_image_even = ct_para.pixel_driven_bp(projector_new, fprjs_even, angles[even_indexes])

                        # # bp for noise
                        recon_custom_noise_odd = ct_para.pixel_driven_bp(projector_new, fnoise_odd, angles[odd_indexes])
                        recon_custom_noise_even = ct_para.pixel_driven_bp(projector_new, fnoise_even, angles[even_indexes])

                        # final recon
                        recon_custom_odd = (recon_custom_image_odd + recon_custom_noise_odd) *2  / 0.019 * 1000 - 1000
                        recon_custom_even = ( recon_custom_image_even + recon_custom_noise_even) *2 / 0.019 * 1000 - 1000

                        recon_noise_odd[:,:,slice_n] = recon_custom_odd
                        recon_noise_even[:,:,slice_n] = recon_custom_even
        
              
            # save recon
            # nb.save(nb.Nifti1Image(img_clean, affine), os.path.join(save_folder_case,'img_clean.nii.gz'))
            if split_angle == False:
                nb.save(nb.Nifti1Image(recon_noise, affine), os.path.join(save_folder_k,'recon.nii.gz'))
                diff = recon_noise - img_clean
                nb.save(nb.Nifti1Image(diff, affine), os.path.join(save_folder_k,'diff.nii.gz'))
            else:
                nb.save(nb.Nifti1Image(recon_noise_odd, affine), os.path.join(save_folder_k,'recon_odd.nii.gz'))
                nb.save(nb.Nifti1Image(recon_noise_even, affine), os.path.join(save_folder_k,'recon_even.nii.gz'))
                # diff_odd = recon_noise_odd - img_clean
                # diff_even = recon_noise_even - img_clean
                # nb.save(nb.Nifti1Image(diff_odd, affine), os.path.join(save_folder_k,'diff_odd.nii.gz'))
                # nb.save(nb.Nifti1Image(diff_even, affine), os.path.join(save_folder_k,'diff_even.nii.gz'))
                # diff_between = recon_noise_odd - recon_noise_even
                # nb.save(nb.Nifti1Image(diff_between, affine), os.path.join(save_folder_k,'diff_odd_even.nii.gz'))
            

L067
/host/d/Data/low_dose_CT/nii_imgs/L067/img.nii.gz
img shape, min, max:  (512, 512, 224) -1024.0 2123.0
spacing:  (2.0, 0.6640625, 0.6640625)
dose factor:  0.23295214508174006
dose factor:  0.33203777120864864
dose factor:  0.34851202551738336
dose factor:  0.24389983082034428
L096
/host/d/Data/low_dose_CT/nii_imgs/L096/img.nii.gz
img shape, min, max:  (512, 512, 330) -1024.0 3071.0
spacing:  (2.0, 0.7421875, 0.7421875)
dose factor:  0.1847992668576043
dose factor:  0.294817655183663
dose factor:  0.3295366114974843
dose factor:  0.1563703776509878
L109
/host/d/Data/low_dose_CT/nii_imgs/L109/img.nii.gz
img shape, min, max:  (512, 512, 128) -1024.0 2220.0
spacing:  (2.0, 0.78125, 0.78125)
dose factor:  0.17190208042311256
dose factor:  0.26524786338966655
dose factor:  0.23143553503645614
dose factor:  0.28641916060592687
L143
/host/d/Data/low_dose_CT/nii_imgs/L143/img.nii.gz
img shape, min, max:  (512, 512, 234) -1024.0 2951.0
spacing:  (2.0, 0.8203125, 0.8203125)
dose factor:  0.2

In [ ]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # HYTH

In [ ]:
# from scipy.ndimage import binary_fill_holes
# i=0
# a = nb.load(os.path.join(patient_sheet[i],'img_no_bed.nii.gz')).get_fdata().astype(np.float32)
# segmented_img = np.zeros_like(a)
# object_mask = np.zeros_like(a)
# for s in range(a.shape[2]):
#     slice_img = a[:,:,s]
#     slice_mask = np.zeros_like(slice_img)
#     slice_mask[slice_img>=-430]= 1
#     slice_mask[slice_img<-430]= 0
#     filled_mask = binary_fill_holes(slice_mask)
#     slice_img_segmented = np.copy(slice_img)
#     slice_img_segmented[filled_mask==0] = np.min(slice_img)
#     object_mask[:,:,s] = filled_mask
#     segmented_img[:,:,s] = slice_img_segmented